# Scrapping data (old code)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen as ureq
import re

In [2]:
df = pd.DataFrame(columns =['price', 'locali', 'superficie', 'bagni','piano'])

In [3]:
links=[]

for i in range(1,20):
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        url = link['href']
        if url.startswith('https://www.immobiliare.it/') and url.endswith('.html'):
            links.append(url)
            scrap_data(url)

for i in range(len(links)):
    content = requests.get(links[i])
    soup=BeautifulSoup(content.text,'html.parser')
    p = soup.find_all('li',class_='features__price')[0].get_text()
    p = p.replace("€", "")
    p = p.replace(" ", "")
    p = p.replace(".", "")
    l1 = Soup.find_all('ul','list-inline list-piped features__list')[0].get_text()
    l1 = l1.replace('m2','')
    numbers = re.compile('\d+(?:\.\d+)?')
    l1 = numbers.findall(l1)
    if len(l1)==4:
         i = [{'price':int(p),'locali': int(l1[0]), 'superficie': int(l1[1]), "bagni": int(l1[2]), "piano": int(l1[3])}]
    print(i)

In [63]:
# content = requests.get(links[0])
# soup = BeautifulSoup(content.text, 'html.parser')
# features = soup.find_all(class_=["list-inline list-piped features__list"])[0].get_text()
# features = features.replace('\xa0', "")
# tmp = features.split("locali")
# locali = int(tmp[0])
# superficie = int(tmp[1].split('m2superficie')[0]) 
# bagni, piano = map(int, tmp[1].split('m2superficie')[1].split()[0].split("bagni"))

# Mongo_DB

In [1]:
from pymongo import MongoClient
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen as ureq
import re

In [3]:
client = MongoClient()
my_key = "AnqLqo5TT9JDfzfqKrDUEYUqJ6HWZJy8"
base = "https://api.mlab.com/api/1"
url_db = base + "/databases?apiKey=" + my_key
response = requests.get(url_db)
databases = json.loads(response.text)
adm_database = databases[0]
adm_database

'adm_hm4'

In [4]:
# structure
headers = {'content-type':'application/json'}

# test data
# flat_data = [{'price':100,'locali': 2, 'superficie': 1, "bagni": 2, "piano": 5}]

In [5]:
# I manually created the collection 'flats' in "adm_hm4" database
collection = 'flats'
col_url = base + '/databases/' + adm_database + "/collections/" + collection +'/?apiKey=' + my_key

# Pushing data into database
# payload = json.dumps(flat_data)
# response = requests.post(col_url, data=payload, headers=headers)

In [6]:
# Checking what is inside our database
response = requests.get(col_url)
result = json.loads(response.text)
result

[]

Now, we have to extract data from websites and push into database

In [7]:
def scrap_data(soup):
    price = soup.find_all('li',class_='features__price')[0].get_text()
    price = price.replace("€", "").replace(" ", "").replace(".", "")
    
    data = soup.find_all('ul','list-inline list-piped features__list')[0].get_text()
    data = data.replace('m2','')
    numbers = re.compile('\d+(?:\.\d+)?')
    data = numbers.findall(data)
    if len(data)==4:
         return [{'price':int(price),'locali': int(data[0]), 'superficie': int(data[1]), "bagni": int(data[2]), "piano": int(data[3])}]

In [8]:
#extractInfo is function to scrap html ; output --> dictionary
import logging

def threadExtract(url):
    logging.info('extracting' + url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    data_dict = scrap_data(soup) #output should be a dictionary
    
    collection = 'flats'
    col_url = base + '/databases/' + adm_database + "/collections/" + collection +'/?apiKey=' + my_key

    # Pushing data into database
    payload = json.dumps(data_dict)
    response = requests.post(col_url, data=payload, headers=headers)

    logging.info("Done extracting")

In [ ]:
links=[]

stop = False
i = 1 
while stop == False:
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, 'html.parser')
    
    for link in soup.find_all('a', href=True):
        url = link['href']
        if url.startswith('https://www.immobiliare.it/') and url.endswith('.html'):
            try:
                threadExtract(url)
                links.append(url)
                if len(links >= 10000): stop = True
            except: continue
    i += 1

In [ ]:
# display what is inside the database
response = requests.get(col_url)
result = json.loads(response.text)
result